# 数据采集

In [14]:
'''
新闻
基本知识：类别，同类别产品
'''

from fake_useragent import UserAgent

ua = UserAgent()
# print(ua.random)
agent = ua.random

headers={"User-Agent":agent}

In [1]:
'''
基本信息，取出中文名、所在行业、概念
1、https://blog.csdn.net/Xidian2850/article/details/103121087?depth_1-utm_source=distribute.pc_relevant.none-task-blog-OPENSEARCH-4&utm_source=distribute.pc_relevant.none-task-blog-OPENSEARCH-4
'''

# url = 'http://vip.stock.finance.sina.com.cn/corp/go.php/vCI_CorpOtherInfo/stockid/002261/menu_num/2.phtml'
# url = 'http://hq.sinajs.cn/list=sh601006'
# url = 'https://hq.sinajs.cn/rn=1587975245746&list=sz002261' # 实时行情
# url = 'http://vip.stock.finance.sina.com.cn/mkt/#chgn_700896' # 概念详情况，股票信息
# http://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/Market_Center.getHQNodeData?page=1&num=100&sort=symbol&asc=1&node=chgn_700896&symbol=&_s_r_a=setlen
# url = 'http://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/Market_Center.getHQNodes'  # 行业，热门概念信息

'''
处理流程：
1、访问http://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/Market_Center.getHQNodes，并保存，得到概念的编码
2、调用http://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/Market_Center.getHQNodeData?page=1&num=200&sort=symbol&asc=1&node=chgn_700896&symbol=&_s_r_a=setlen
   替换概念代码得到所有编码
'''






'\n处理流程：\n1、访问http://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/Market_Center.getHQNodes，并保存，得到概念的编码\n2、调用http://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/Market_Center.getHQNodeData?page=1&num=200&sort=symbol&asc=1&node=chgn_700896&symbol=&_s_r_a=setlen\n   替换概念代码得到所有编码\n'

In [10]:
??requests.get

# 关系

## 基于类别关系 

In [30]:
'''
创建实体，关系
实体关系
1、概念（如：在线教育）：包括（Include）：股票 （属性：代码、总股本、近一个月平均价）
2、股票主：股价相关性（属性：正向，关联系数）：股票副
'''

from py2neo import Graph
from root import get_root_path
from lib.utils.sdata_gather import StockDataGather
import os
graph = Graph(uri='bolt://127.0.0.1:7687',auth=('neo4j','654321')) 
# 新建概念信息
# create_concept = 'CREATE (OnlineEducation:Concept {title:\'OnlineEducation\',name:\'在线教育\', code:\'' + 'chgn_700185' + '\'})'
# graph.run(create_concept)

# 新建该概念下的股票信息
create_stock = '''CREATE (Stock%s:Stock {name:'%s', code:'%s'})'''
# create_stock_r_concept = 'CREATE (OnlineEducation) - [:CONTAIN] - (Stock%s)'
create_stock_r_concept = '''MATCH (K1:Concept {title:'OnlineEducation'}), (K2:Stock {code:'%s'}) CREATE (K1) - [:CONTAIN] -> (K2)'''

gather = StockDataGather(os.path.sep.join([get_root_path(),'data']))
stocks = gather.get_concept_stocks('在线教育')
# print(len(stocks))
for item in stocks:
    code = item['symbol'].replace('sh','').replace('sz','')
    crs = create_stock % (code,item['name'],code)
    cr_r_s = create_stock_r_concept % (code)
    graph.run(crs)
    graph.run(cr_r_s)

print('create over ')


create over 


In [10]:
# https://blog.csdn.net/sinat_26917383/article/details/79901207 py2neo查询方式
from py2neo import Graph
from root import get_root_path
from lib.utils.sdata_gather import StockDataGather
import os
graph = Graph(uri='bolt://127.0.0.1:7687',auth=('neo4j','654321')) 
rel_query = '''MATCH (C:Concept {name:'%s'}) -[:CONTAIN] -> (S:Stock) RETURN S'''
detail = graph.run(rel_query % ('在线教育')).data()
print(type(detail), detail.size())
# print(detail.iloc(0,0))
# print(detail[0])
# print(detail[0].get('S').get('code'))
# for item in detail:
#     print(item['S'].get('code'),item['S'].get('name'))

TypeError: 'numpy.int32' object is not callable

## 基于数据关系 